In [2]:
import pandas as pd
import torch
from transformers import pipeline
from datasets import Dataset
import pandas as pd
from tqdm.auto import tqdm

In [3]:
folder_path= 'E:\\Case Comp\\NEST\\Training\\'
# usecase_3.to_parquet(parquet_file_path, index=False)

# Load the Parquet file back into a DataFrame
usecase_3 = pd.read_csv(folder_path + 'usecase_3_.csv', index_col=False)

usecase_3.head()

,Unnamed: 0.1,Unnamed: 0,NCT Number,Study Title,Study URL,Acronym,Study Status,Brief Summary,Study Results,Conditions,...,Study Design,Other IDs,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents
0,75209,118014,NCT00559130,Efficacy Study of CytoSorb Hemoperfusion Devic...,https://clinicaltrials.gov/study/NCT00559130,NaN,COMPLETED,The hypothesis of this study is use of CytoSor...,NO,Acute Respiratory Distress Syndrome|Acute Lung...,...,Allocation: RANDOMIZED|Intervention Model: PAR...,2007-01,2007-11,2011-04,2011-06,2007-11-16,NaN,2011-06-07,"Aachen, Germany|Berlin, Germany|Bonn, Germany|...",NaN
1,6356,9987,NCT00937664,Safety and Tolerability Study of AZD7762 in Co...,https://clinicaltrials.gov/study/NCT00937664,NaN,TERMINATED,The primary purpose of this study is to find o...,NO,Cancer|Solid Tumors|Advanced Solid Malignancies,...,Allocation: NON_RANDOMIZED|Intervention Model:...,D1040C00008,2009-07,2011-02,2011-02,2009-07-13,NaN,2011-02-07,"Research Site, Minami-ku, Fukuoka, Japan",NaN
2,143427,226012,NCT00441597,Does Atorvastatin Reduce Ischemia-Reperfusion ...,https://clinicaltrials.gov/study/NCT00441597,NaN,COMPLETED,To study the impact of 3 day exposure to atorv...,NO,Ischemia Reperfusion Injury|Cardiovascular Dis...,...,Allocation: RANDOMIZED|Intervention Model: CRO...,atorv01,2007-02,2009-02,2009-03,2007-03-01,NaN,2009-03-17,"Radboud University Nijmegen Medical Centre, Ni...",NaN
3,138885,218952,NCT03296228,Comparison of Dynamic Radiographs in Determini...,https://clinicaltrials.gov/study/NCT03296228,NaN,COMPLETED,The purpose of this study is to identify the f...,NO,Adolescent Idiopathic Scoliosis,...,Observational Model: |Time Perspective: p,UW 16-208,2016-05-01,2018-05-01,2018-12-31,2017-09-28,NaN,2020-05-06,"Duchess of Kent Children's Hospital, Hong Kong...",NaN
4,9769,15382,NCT00421603,A Placebo-Controlled Study of Mixed Amphetamin...,https://clinicaltrials.gov/study/NCT00421603,TACT,COMPLETED,"The proposed protocol is a double-blind, place...",YES,Cocaine Dependence,...,Allocation: RANDOMIZED|Intervention Model: PAR...,#5368|R01DA022217,2007-02,2010-05,2010-05,2007-01-12,2013-02-28,2019-04-24,"STARS, New York, New York, 10032, United States",NaN


In [4]:
usecase_3 = usecase_3.head(500)

In [5]:
# ✅ Use GPU if available
device = 0 if torch.cuda.is_available() else -1
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

# ✅ Define categories
categories = [
    "Oncology", "Non-Oncology"
]

# ✅ Convert Pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(usecase_3[['Conditions']])

# ✅ Batch processing function (Uses GPU)
def batch_classify(batch, threshold=0.7):
    """Classifies a batch of conditions into predefined categories."""
    results = classifier(batch['Conditions'], candidate_labels=categories, batch_size=64)  # ✅ Optimized batch inference

    classified_labels = []
    for classification in results:
        best_label = classification['labels'][0]
        best_score = classification['scores'][0]
        classified_labels.append(best_label if best_score >= threshold else "Other Rare or Unclassified")

    return {"Conditions_Category": classified_labels}

# ✅ Enable Live Progress Bar Updates
batch_size = 512
num_batches = (len(dataset) // batch_size) + 1

classified_labels = []
progress_bar = tqdm(range(num_batches), desc="Processing Conditions", unit="batch", leave=True, mininterval=0.1, ncols=100)

for i in progress_bar:
    batch_data = dataset[i * batch_size:(i + 1) * batch_size]
    if batch_data:  # Ensure batch is not empty
        result = batch_classify(batch_data)
        classified_labels.extend(result["Conditions_Category"])
        progress_bar.set_postfix(rows_processed=len(classified_labels))  # ✅ Live row count update
        progress_bar.update(1)  # ✅ Ensures progress bar moves forward in real time

# ✅ Convert Back to Pandas DataFrame
usecase_3['Conditions_Category'] = classified_labels

# ✅ Show Completion Status
total_rows = len(usecase_3)
classified_rows = usecase_3['Conditions_Category'].notnull().sum()
completion_percentage = (classified_rows / total_rows) * 100
tqdm.write(f"✅ Classification Completed: {completion_percentage:.2f}% of rows processed.")

# ✅ Display Sample Results
tqdm.write(str(usecase_3[['Conditions', 'Conditions_Category']].head()))
# usecase_3.to_csv('category_updated.csv')

Processing Conditions: 100%|██████████████████| 1/1 [04:13<00:00, 253.65s/batch, rows_processed=500]

✅ Classification Completed: 100.00% of rows processed.
                                          Conditions  \
0  Acute Respiratory Distress Syndrome|Acute Lung...   
1    Cancer|Solid Tumors|Advanced Solid Malignancies   
2  Ischemia Reperfusion Injury|Cardiovascular Dis...   
3                    Adolescent Idiopathic Scoliosis   
4                                 Cocaine Dependence   

          Conditions_Category  
0  Other Rare or Unclassified  
1                    Oncology  
2  Other Rare or Unclassified  
3                Non-Oncology  
4                Non-Oncology  


In [6]:
usecase_3.to_csv(folder_path +'category_updated_train.csv')

In [7]:
usecase_3.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'NCT Number', 'Study Title', 'Study URL',
       'Acronym', 'Study Status', 'Brief Summary', 'Study Results',
       'Conditions', 'Interventions', 'Primary Outcome Measures',
       'Secondary Outcome Measures', 'Other Outcome Measures', 'Sponsor',
       'Collaborators', 'Sex', 'Age', 'Phases', 'Enrollment', 'Funder Type',
       'Study Type', 'Study Design', 'Other IDs', 'Start Date',
       'Primary Completion Date', 'Completion Date', 'First Posted',
       'Results First Posted', 'Last Update Posted', 'Locations',
       'Study Documents', 'Conditions_Category'],
      dtype='object')